# Применение моделей машинного обучения для обнаружения сетевых атак

In [ ]:
import pandas as pd
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras import regularizers
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics

Описание датасета https://www.unb.ca/cic/datasets/nsl.html


Файл с данными:
https://drive.google.com/file/d/19SK6HfVjDTmvDB82mqpLjJPWmd7XHrVv/view?usp=sharing

## Загрузка данных

Загрузите датасет и получите информацию о нем.

In [1]:
df1 = ...

In [ ]:
def pie_plot(df, cols_list, rows, cols):
    fig, axes = plt.subplots(rows, cols)
    for ax, col in zip(axes.ravel(), cols_list):
        df[col].value_counts().plot(ax=ax, kind='pie', figsize=(15, 15), fontsize=10, autopct='%1.0f%%')
        ax.set_title(str(col), fontsize = 12)
    plt.show()
pie_plot(df1, ['protocol_type', 'outcome'], 1, 2)

## Предварительная обработка данных

In [ ]:
df1.loc[df1['outcome'] == "normal", "outcome"] = 'normal'
df1.loc[df1['outcome'] != 'normal', "outcome"] = 'attack'

In [ ]:
df1['level'].unique(), df1['outcome'].unique()

(array([20, 15, 19, 21, 18, 17, 16, 12, 14, 11,  2, 13, 10,  9,  8,  7,  3,
         5,  1,  6,  0,  4]),
 array(['normal', 'attack'], dtype=object))

In [ ]:
def Scaling(df_num, cols):
    std_scaler = RobustScaler()
    ...
    std_df = ...
    return std_df

In [ ]:
cat_cols = ['is_host_login','protocol_type','service','flag','land', 'logged_in','is_guest_login', 'level', 'outcome']
def preprocess(dataframe):
    df_num = dataframe.drop(cat_cols, axis=1)
    num_cols = df_num.columns
    scaled_df = Scaling(df_num, num_cols)

    dataframe.drop(labels=num_cols, axis="columns", inplace=True)
    dataframe[num_cols] = scaled_df[num_cols]

    dataframe.loc[dataframe['outcome'] == "normal", "outcome"] = 0
    dataframe.loc[dataframe['outcome'] != 0, "outcome"] = 1

    dataframe = pd.get_dummies(dataframe, columns = ['protocol_type', 'service', 'flag'])
    return dataframe

In [ ]:
df2 = preprocess(df1)

In [ ]:
df2.describe().style.background_gradient(cmap='Blues').set_properties(**{'font-family':'Segoe UI'})

## Анализ главных компонент

Анализ главных компонент, или PCA, — это статистический метод преобразования данных высокой размерности в данные низкой размерности путем выбора наиболее важных признаков, которые содержат максимальную информацию о наборе данных. Признаки выбираются на основе дисперсии, которую они вызывают в выходных данных. Признак, который вызывает самую высокую дисперсию, является первым главным компонентом. Признак, который отвечает за вторую по величине дисперсию, считается вторым главным компонентом и т. д. Важно отметить, что главные компоненты не имеют никакой корреляции друг с другом.

Преимущества PCA
Существует два основных преимущества снижения размерности с помощью PCA.

Время обучения алгоритмов значительно сокращается при меньшем количестве признаков.
Не всегда возможно анализировать данные в высоких размерностях. Например, если в наборе данных 100 признаков. Общее количество диаграмм рассеяния, необходимых для визуализации данных, составит 100(100-1)2 = 4950. На практике анализировать данные таким образом невозможно.


In [ ]:
X = df2.drop(['outcome', 'level'] , axis = 1).values
y = df2['outcome'].values

pca = PCA(n_components=20)
pca = ...
x_reduced = pca.transform(X)
print("Number of original features is {} and of reduced features is {}".format(X.shape[1], x_reduced.shape[1]))

 ## Разделение данных на обучающую и тестовую выборки

In [ ]:
y = y.astype('int')
x_train, x_test, y_train, y_test = ...
x_train_reduced, x_test_reduced, y_train_reduced, y_test_reduced = ...

In [ ]:
x_train_reduced.shape, x_train.shape

In [ ]:
y_train_reduced.shape, y_train.shape

## Модель нейронной сети для задачи классификации

Слои

Плотный слой: полностью связанный слой, в котором каждый нейрон соединяется с каждым нейроном в следующем слое.
Сверточный слой: необходим для обработки данных изображений, захвата пространственных иерархий.
Рекуррентный слой: подходит для последовательных данных, таких как текст и временные ряды, позволяя сети сохранять информацию с течением времени.

Компиляция

Функции потерь: измеряют, насколько хорошо работает модель. Общие функции потерь включают binary_crossentropy, categorical_crossentropy для классификации и mean_squared_error для регрессии.

Оптимизаторы: алгоритмы для обновления весов модели на основе функции потерь. Популярные оптимизаторы включают Adam, SGD и RMSprop.

Метрики: используются для оценки производительности модели во время обучения и тестирования, например, точности.

Обучение

Fit: обучает модель на обучающих данных. Ключевые параметры включают:
Размер пакета: количество образцов, обработанных до обновления модели.
Эпохи: количество полных проходов через обучающий набор данных.

**Подберите архитектуру и гиперпараметры полносвязной нейронной сети для достижения accuracy >= 0.98**

Используем данные, которые мы получили после понижения размерности (__reduced)

In [ ]:
x_train_reduced = tf.convert_to_tensor(x_train_reduced)
y_train_reduced = tf.convert_to_tensor(y_train_reduced)

# Build the model
model = Sequential()

model.add(Dense(units=64, activation='relu', input_shape=...))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train_reduced, y_train_reduced,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)


model.summary()



In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.legend()
plt.grid(True)

Постройте график
Sparse Categorical Crossentropy Loss vs Epoch

In [2]:
...

Ellipsis

Примените вашу модель на тестовых данных

In [3]:
predicted = ...

Постройте confusion matrix

In [5]:
confusion_matrix = ...

Выведете все метрики качества, сделайте выводы

In [6]:
...

Ellipsis

**Обучите вашу модель на выборке x_train, x_test, y_train, y_test и проделайте те же шаги, которые сделаны выше**